## Imports ##

In [1]:
import pandas as pd
import numpy as np
# import torch

## Feature engineering ##

In [ ]:
def create_features(current_meet, previous_meet):
    features = current_meet.copy()

    features['prev_squat'] = previous_meet['Best3SquatKg'].iloc[-1]
    features['prev_bench'] = previous_meet['Best3BenchKg'].iloc[-1]
    features['prev_deadlift'] = previous_meet['Best3DeadliftKg'].iloc[-1]

    features['avg_squat'] = previous_meet['Best3SquatKg'].mean()
    features['avg_bench'] = previous_meet['Best3BenchKg'].mean()
    features['avg_deadlift'] = previous_meet['Best3DeadliftKg'].mean()
    
    features['days_since_last_meet'] = (
        pd.to_datetime(current_meet['Date']) - pd.to_datetime(previous_meet['Date'].iloc[-1])
    ).days
    features['total_meets'] = len(previous_meet)
    
    # total kg lifted to bodyweight ratio on previous meet
    features['total_bodyweight_ratio'] = previous_meet['TotalKg'].iloc[-1] / previous_meet['BodyweightKg'].iloc[-1]
    
    if len(previous_meet) >= 2:
        features['percent_gain_since_last'] = (
            (previous_meet['TotalKg'].iloc[-1] - previous_meet['TotalKg'].iloc[-2]) / 
            previous_meet['TotalKg'].iloc[-2]
        )
        features['avg_improvement_rate'] = (
            (previous_meet['TotalKg'].iloc[-1] - previous_meet['TotalKg'].iloc[0]) / 
            (len(previous_meet) - 1) / previous_meet['TotalKg'].iloc[0]
        )
        features['bodyweight_change'] = previous_meet['BodyweightKg'].iloc[-1] - previous_meet['BodyweightKg'].iloc[-2]
    else:
        features['percent_gain_since_last'] = 0
        features['avg_improvement_rate'] = 0
        features['bodyweight_change'] = 0
    
    features['total_std'] = previous_meet['TotalKg'].std() if len(previous_meet) > 1 else 0
    
    return features

## Create features for one lifter and all lifters (run above first to confirm features to engineer)

In [27]:
def process_single_lifter(lifter_data):
    lifting_data = []

    for i in range(1, len(lifter_data)):
        current = lifter_data.iloc[i]
        previous = lifter_data.iloc[:i]
        features = create_features(current, previous)
        lifting_data.append(features)
    return pd.DataFrame(lifting_data)

def engineer_features(df, meets=2):
    print(f'Total lifters: {df["Name"].nunique()}')

    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(['Name', 'Date']).reset_index(drop=True) # sort by name, date
    all_lifting_data = []

    for name, lifter_data in df.groupby('Name'): # for each lifters competition history
        if len(lifter_data) < meets: # only can predict lifters with at least two comp history
            continue
        features = process_single_lifter(lifter_data)
        all_lifting_data.append(features)

    result = pd.concat(all_lifting_data)

    print(f"\nFeature engineering complete!")
    print(f"Training examples created: {len(result)}")
    print(f"{result['Name'].nunique()} lifters with 2+ meets")
    return result

df = pd.read_csv('../data/2-preprocessed/openpowerlifting-IPF-clean_MIN.csv')

df_with_features = engineer_features(df)
df_with_features.to_csv('../data/3-features/IPF_features_train_test.csv', index=False)



Total lifters: 158815

Feature engineering complete!
Training examples created: 212875
73582 lifters with 2+ meets


## DATASET SPLIT

In [5]:
df_with_features = pd.read_csv('../data/3-features/IPF_features_train_test.csv')
# sort by date, to prepare for training
df_with_features['Date'] = pd.to_datetime(df_with_features['Date'])
df_with_features = df_with_features.sort_values('Date').reset_index(drop=True)

split_dataset = df_with_features['Date'].quantile(0.8)

train_df = df_with_features[df_with_features['Date'] < split_dataset].copy()
test_df = df_with_features[df_with_features['Date'] >= split_dataset]

print(f'train {train_df["Date"].min()} - {train_df["Date"].max()}')
print(f'test {test_df["Date"].min()} - {test_df["Date"].max()}')

train 1967-02-26 00:00:00 - 2024-02-24 00:00:00
test 2024-02-29 00:00:00 - 2025-09-21 00:00:00


## SELECT FEATURES, ENCODING, IMPUTATION

In [9]:
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

feature_columns = [
    'prev_squat', 'prev_bench', 'prev_deadlift', 
    'avg_squat', 'avg_bench', 'avg_deadlift',
    'days_since_last_meet', 'total_meets',
    'total_bodyweight_ratio',
    'percent_gain_since_last', 'bodyweight_change', #'avg_improvement_rate', 'total_std',
    # 'Sex_M', 'Sex_F',
]

# label/ordinal encoding can lead to model learning that there's a relationship between these categories, 
# as these sex/gender has no relationship, avoid using to prevent this
# but this works on xgboosts with 'enable_categorical'
train_df['Sex'] = train_df['Sex'].map({'M': 1, 'F': 0})
feature_columns.append('Sex')
train_df['Sex'].astype('category')

# imputation methods
# print(train_df[train_df['BodyweightKg'].isnull()])

imputer = SimpleImputer(strategy='median')
train_df['BodyweightKg'] = imputer.fit_transform(train_df[['BodyweightKg']])
print(imputer.statistics_)

'''BUCKETIZE AGE WITH DIVISION GROUPS??'''
# drop age, too many missing values for imputation, not using for training
# train_df = train_df.drop(columns=['Age'])

# fill na with 0? over 30k rows of missing age
# print(train_df.isnull().sum())
# train_df[feature_columns] = train_df[feature_columns].fillna(0)

correlation = train_df.corr(numeric_only=True)
correlation['TotalKg'].sort_values(ascending=False)


[81.35]


TotalKg                    1.000000
Best3SquatKg               0.979270
Best3DeadliftKg            0.974053
Best3BenchKg               0.953953
avg_squat                  0.944920
prev_squat                 0.943451
avg_deadlift               0.940946
prev_deadlift              0.938767
prev_bench                 0.928995
avg_bench                  0.922490
BodyweightKg               0.690258
total_bodyweight_ratio     0.635875
total_std                  0.266872
total_meets                0.156892
bodyweight_change          0.069051
days_since_last_meet       0.043096
percent_gain_since_last   -0.016940
avg_improvement_rate      -0.018367
Age                       -0.059618
Sex                             NaN
Name: TotalKg, dtype: float64

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBRegressor

# TIME SERIES CROSS VALIDATIONS
X = train_df[feature_columns]
y = train_df['TotalKg']
scores = {'mae': [], 'rmse': [], 'mse': [], 'r2': []}

tscv = TimeSeriesSplit(n_splits=5)
for index, (train_index, test_index) in enumerate(tscv.split(X)):
    X_train, X_validate = X.iloc[train_index], X.iloc[test_index]
    y_train, y_validate = y.iloc[train_index], y.iloc[test_index]

    # gbr = XGBRegressor(
    #     enable_categorical=True, 
    #     max_depth=5, 
    #     learning_rate=0.3, 
    #     n_estimators=200, 
    #     subsample=0.8, 
    #     colsample_bytree=0.8,
    #     gamma = 0,
    #     random_state = 42
    #     )
    gbr = XGBRegressor(
        enable_categorical=True, 
        max_depth=8, 
        learning_rate=0.1, 
        n_estimators=1500, 
        subsample=1, 
        colsample_bytree=0.9,
        reg_lambda=100,
        reg_alpha=0.15,
        gamma=1,
        random_state = 42
        )
    gbr.fit(X_train, y_train)
    gbr_y_pred = gbr.predict(X_validate)
    # importance = gbr.feature_importances_

    mae = mean_absolute_error(y_validate, gbr_y_pred)
    rmse = root_mean_squared_error(y_validate, gbr_y_pred)
    mse = mean_squared_error(y_validate, gbr_y_pred)
    r2 = r2_score(y_validate, gbr_y_pred)
    scores['mae'].append(mae)
    scores['rmse'].append(rmse)
    scores['mse'].append(mse)
    scores['r2'].append(r2)
    print(f'Fold {index}, MAE={mae:.2f}, RMSE={rmse:.2f}, MSE={mse:.2f}, R2={r2:.3f}')

# sorted_importance = np.argsort(importance)[::-1]
# x = range(len(importance))
# labels = np.array(feature_columns)[sorted_importance]
# plt.bar(x, importance[sorted_importance], tick_label=labels)
# plt.xticks(rotation=90)
# plt.show()


Fold 0, MAE=23.41, RMSE=38.27, MSE=1464.59, R2=0.940
Fold 1, MAE=22.93, RMSE=38.14, MSE=1454.66, R2=0.940
Fold 2, MAE=23.44, RMSE=38.65, MSE=1493.44, R2=0.937
Fold 3, MAE=24.89, RMSE=40.03, MSE=1602.56, R2=0.932
Fold 4, MAE=22.04, RMSE=37.30, MSE=1391.21, R2=0.940


In [ ]:
from sklearn.model_selection import GridSearchCV

print(XGBRegressor().get_params)

param_grid = {
    'max_depth': [3, 5, 7],
    'subsample': [0.5, 0.7, 1],
    'learning_rate': [0.1, 0.2, 0.3], 
    'n_estimators': np.arange(200, 2000, 200), 
    'colsample_bytree': [0.9],
    'reg_lambda': [100],
    'reg_alpha': [0.15],
    'gamma': [1],
}

xgb_model = XGBRegressor()
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X, y)

print("Best set of HP: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_dist = {
    'n_estimators': np.arange(200, 2000, 100),
    'max_depth': [3, 4, 5, 6, 7, 8, 10],
    'learning_rate': np.logspace(-3, 0, 20),     # 0.001 → 1.0
    'subsample': np.linspace(0.5, 1.0, 6),
    'colsample_bytree': np.linspace(0.5, 1.0, 6),
    'gamma': np.linspace(0, 5, 6),
    'reg_lambda': np.logspace(-3, 2, 10),
    'reg_alpha': np.logspace(-3, 2, 10)
}

xgb = XGBRegressor()

random_search = RandomizedSearchCV(
    xgb,
    param_distributions=param_dist,
    n_iter=100,                  # tries 100 combos → far better than any grid!
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=2,
)

random_search.fit(X, y)

print(random_search.best_params_)